# Data Crawling

In [1]:
!dpkg --configure -a

In [2]:
# write your import and necessary web crawling libary here
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

     |████████████████████████████████| 954 kB 5.1 MB/s 
     |████████████████████████████████| 138 kB 52.7 MB/s 
     |████████████████████████████████| 356 kB 61.1 MB/s 
     |████████████████████████████████| 3.5 MB 38.8 MB/s 
     |████████████████████████████████| 55 kB 2.6 MB/s 
     |████████████████████████████████| 54 kB 2.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.7 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidi

In [3]:

# write your import and necessary web crawling libary here
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from time import sleep
import sys

import csv
import pandas as pd
import numpy as np
from csv import writer
import numpy as np
import selenium
from calendar import monthrange
import time

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
hour_values_1 = ["12:00 am", "1:00 am", "2:00 am", "3:00 am", "4:00 am", "5:00 am", "6:00 am",
              "7:00 am", "8:00 am", "9:00 am", "10:00 am", "11:00 am", "12:00 pm",
              "1:00 pm", "2:00 pm", "3:00 pm", "4:00 pm", "5:00 pm", "6:00 pm",
              "7:00 pm", "8:00 pm", "9:00 pm", "10:00 pm", "11:00 pm"
              ]
hour_values_2 = [f"0{i}:00" for i in range(10)] + [f"{i}:00" for i in range(10, 24)]
hour_values = hour_values_1 + hour_values_2
hour_dic = {}
for i in range(24):
  hour_dic[hour_values_1[i]] = i
  hour_dic[hour_values_2[i]] = i

In [ ]:
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')


In [ ]:
driver= webdriver.Chrome('chromedriver',chrome_options=chrome_options)
driver.get("https://www.timeanddate.com/weather/australia/melbourne/historic?month=1&year=2019")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: use options instead of chrome_options
  """Entry point for launching an IPython kernel.


In [ ]:
hour = driver.find_element_by_xpath(f'//*[@id="wt-his"]/tbody/tr[{1}]/th').text.split("\n")[0]
hour 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  """Entry point for launching an IPython kernel.


'12:00 am'

In [ ]:
def year_weather(year, month_start, month_end):
    driver= webdriver.Chrome('chromedriver',chrome_options=chrome_options)
    headers = ["DateTime", "Time", "Temperature", "Weather", "Wind Speed", "Humidity", "Barometer", "Visibility"]
    ds = pd.DataFrame(columns = headers)
    start_time = time.time()
    for month_value in range(month_start,month_end+1):
        print(month_value)
        url = f"https://www.timeanddate.com/weather/australia/melbourne/historic?month={month_value}&year={year}"
        driver.get(url)
        for day in range(1, monthrange(year, month_value)[1]+1):
            driver.find_element_by_xpath(f'/html/body/div[6]/main/article/div[6]/div[3]/a[{day}]').click()
            i = 1
            while True:
                try:
                    hour = driver.find_element_by_xpath(f'//*[@id="wt-his"]/tbody/tr[{i}]/th').text.split("\n")[0]
                    if hour in hour_values:
                        record = {
                            "DateTime": "-".join([str(year), str(month_value), str(day)]),
                            "Time": hour,
                            "Temperature": driver.find_element_by_xpath(f'//*[@id="wt-his"]/tbody/tr[{i}]/td[2]').text,
                            "Weather": driver.find_element_by_xpath(f'//*[@id="wt-his"]/tbody/tr[{i}]/td[3]').text.split(".")[0],
                            "Wind Speed": driver.find_element_by_xpath(f'//*[@id="wt-his"]/tbody/tr[{i}]/td[4]').text,
                            "Humidity": driver.find_element_by_xpath(f'//*[@id="wt-his"]/tbody/tr[{i}]/td[6]').text,
                            "Barometer": driver.find_element_by_xpath(f'//*[@id="wt-his"]/tbody/tr[{i}]/td[7]').text,
                            "Visibility": driver.find_element_by_xpath(f'//*[@id="wt-his"]/tbody/tr[{i}]/td[8]').text
                            }
                        ds = ds.append(record, ignore_index=True)
                    i += 1
                except:
                    break
    print("--- %s seconds ---" % (time.time() - start_time))
    return ds

In [ ]:
weather_data_2019 = year_weather(2019, 11, 12)
weather_data_2019.to_csv("drive/MyDrive/SIT782/data/weather_2019.csv", index=False)

In [ ]:
weather_data_2020 = year_weather(2020, 1, 12)
weather_data_2020.to_csv("drive/MyDrive/SIT782/data/weather_2020.csv", index=False)

In [ ]:
weather_data_2021 = year_weather(2021, 1, 10)
weather_data_2021.to_csv("drive/MyDrive/SIT782/data/weather_2021.csv", index=False)

# Data Cleaning

In [ ]:
weather_data_2019 = pd.read_csv("drive/MyDrive/SIT782/data/weather_2019.csv")
weather_data_2020 = pd.read_csv("drive/MyDrive/SIT782/data/weather_2020.csv")
weather_data_2021 = pd.read_csv("drive/MyDrive/SIT782/data/weather_2021.csv")

In [ ]:
weather_data = weather_data_2019.append(weather_data_2020)
weather_data = weather_data.append(weather_data_2021)
weather_data = weather_data.reset_index(drop=True)

In [ ]:
weather_data

,DateTime,Time,Temperature,Weather,Wind Speed,Humidity,Barometer,Visibility
0,2019-11-1,1:00 am,70 °F,Clear,10 mph,38%,"29.83 ""Hg",10 mi
1,2019-11-1,2:00 am,68 °F,Clear,10 mph,40%,"29.80 ""Hg",10 mi
2,2019-11-1,3:00 am,68 °F,Clear,12 mph,40%,"29.80 ""Hg",10 mi
3,2019-11-1,4:00 am,68 °F,Clear,8 mph,46%,"29.77 ""Hg",10 mi
4,2019-11-1,5:00 am,68 °F,Clear,8 mph,46%,"29.80 ""Hg",10 mi
...,...,...,...,...,...,...,...,...
16873,2021-10-31,7:00 pm,63 °F,Sunny,10 mph,56%,"30.18 ""Hg",10 mi
16874,2021-10-31,8:00 pm,59 °F,Clear,8 mph,63%,"30.21 ""Hg",10 mi
16875,2021-10-31,9:00 pm,55 °F,Clear,5 mph,72%,"30.24 ""Hg",10 mi
16876,2021-10-31,10:00 pm,54 °F,Clear,6 mph,77%,"30.24 ""Hg",10 mi


In [ ]:
#fix datatime
#from 2019-1-1 to 2019-01-01
month = [f"{i}" for i in range(1,13)]
month_fix = [f"0{i}" for i in range(1,10)] + [f"{i}" for i in range(10,13)]
month_dic = {}
for i in range(12):
  month_dic[month[i]] = month_fix[i]

day = [f"{i}" for i in range(1,32)]
day_fix = [f"0{i}" for i in range(1,10)] + [f"{i}" for i in range(10,32)]
day_dic = {}
for i in range(31):
  day_dic[day[i]] = day_fix[i]

#fix date
#from 1:00 am to 1
hour_values_1 = ["12:00 am", "1:00 am", "2:00 am", "3:00 am", "4:00 am", "5:00 am", "6:00 am",
              "7:00 am", "8:00 am", "9:00 am", "10:00 am", "11:00 am", "12:00 pm",
              "1:00 pm", "2:00 pm", "3:00 pm", "4:00 pm", "5:00 pm", "6:00 pm",
              "7:00 pm", "8:00 pm", "9:00 pm", "10:00 pm", "11:00 pm"
              ]
hour_values_2 = [f"0{i}:00" for i in range(10)] + [f"{i}:00" for i in range(10, 24)]
hour_values = hour_values_1 + hour_values_2
hour_dic = {}
for i in range(24):
  hour_dic[hour_values_1[i]] = i
  hour_dic[hour_values_2[i]] = i

def fix_date(date):
  y, m, d = date.split("-")
  m = month_dic[m]
  d = day_dic[d]
  return "-".join([y,m,d])


def fix_time(time):
  return hour_dic[time]

#Fix temperate from  °F	to  °C
def fix_temp(temp):
  temp = int(temp[:-2])
  return round((temp - 32) * 5/9)

#Fix win speed
def fix_wind_speed(speed):
  if "No" in str(speed):
    return 0
  if str(speed) == "nan":
    return np.nan
  return int(speed[:-4])

#Fix Humidity
def fix_humidity(percent):
  if percent != percent:
    return np.nan
  return int(percent[:-1])

def fix_barometer(barometer):
  return float(barometer[:-4])

def fix_visibility(mile):
  if mile != mile:
    return np.nan
  return float(mile[:-3])

In [ ]:
def weather_cleaning(weather_data):
    weather_data["DateTime"] =   weather_data["DateTime"].map(fix_date)
    weather_data["Time"] =  weather_data["Time"].map(fix_time)
    weather_data["Temperature"] = weather_data["Temperature"].map(fix_temp)
    weather_data["Wind Speed"] = weather_data["Wind Speed"].map(fix_wind_speed)
    weather_data["Humidity"] = weather_data["Humidity"].map(fix_humidity)
    weather_data["Barometer"]=  weather_data["Barometer"].map(fix_barometer)
    weather_data["Visibility"] =  weather_data["Visibility"].map(fix_visibility)
    return weather_data

In [ ]:
weather_data = weather_cleaning(weather_data)

In [13]:
weather_data

,DateTime,Time,Temperature,Weather,Wind Speed,Humidity,Barometer,Visibility
0,2019-11-01,1,21,Clear,10.0,38.0,29.83,10.0
1,2019-11-01,2,20,Clear,10.0,40.0,29.80,10.0
2,2019-11-01,3,20,Clear,12.0,40.0,29.80,10.0
3,2019-11-01,4,20,Clear,8.0,46.0,29.77,10.0
4,2019-11-01,5,20,Clear,8.0,46.0,29.80,10.0
...,...,...,...,...,...,...,...,...
16873,2021-10-31,19,17,Sunny,10.0,56.0,30.18,10.0
16874,2021-10-31,20,15,Clear,8.0,63.0,30.21,10.0
16875,2021-10-31,21,13,Clear,5.0,72.0,30.24,10.0
16876,2021-10-31,22,12,Clear,6.0,77.0,30.24,10.0


In [14]:
#Check Missing Values
weather_data.isnull().sum()

DateTime           0
Time               0
Temperature        0
Weather            4
Wind Speed         1
Humidity           2
Barometer          0
Visibility     10638
dtype: int64

In [15]:
#Fill missing values with previous values
weather_data.fillna(method='ffill', inplace=True)

In [17]:
#Check Missing Values agian
weather_data.isnull().sum()

DateTime       0
Time           0
Temperature    0
Weather        0
Wind Speed     0
Humidity       0
Barometer      0
Visibility     0
dtype: int64

In [19]:
weather_data

,DateTime,Time,Temperature,Weather,Wind Speed,Humidity,Barometer,Visibility
0,2019-11-01,1,21,Clear,10.0,38.0,29.83,10.0
1,2019-11-01,2,20,Clear,10.0,40.0,29.80,10.0
2,2019-11-01,3,20,Clear,12.0,40.0,29.80,10.0
3,2019-11-01,4,20,Clear,8.0,46.0,29.77,10.0
4,2019-11-01,5,20,Clear,8.0,46.0,29.80,10.0
...,...,...,...,...,...,...,...,...
16873,2021-10-31,19,17,Sunny,10.0,56.0,30.18,10.0
16874,2021-10-31,20,15,Clear,8.0,63.0,30.21,10.0
16875,2021-10-31,21,13,Clear,5.0,72.0,30.24,10.0
16876,2021-10-31,22,12,Clear,6.0,77.0,30.24,10.0


In [18]:
#Save CSV
weather_data.to_csv("drive/MyDrive/SIT782/data/hourly_weather_data.csv", index=False)